In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
from plot_config import *

In [3]:
from utils import *

In [ ]:
ds = open_domain_all()

In [ ]:
limit = 1825

In [ ]:
just_for_time = '/sciserver/filedb10-01/ocean/wenrui_temp/vol_budget'

rtime = np.array(xr.open_zarr(just_for_time).time)

In [ ]:
%%time
terms = {}
for var in rhs_list:
    terms[var] = [0]
for i in range(1,len(rtime),1):
    print(i,end = ' ')
    length = np.minimum(i+1,limit)
    stupid_xr_vec = xr.DataArray(np.arange(length), dims = 'stupid')
    for var in rhs_list:
        slc = ds[var][i-stupid_xr_vec,stupid_xr_vec]
        terms[var].append(np.nansum(slc))

In [ ]:
indices = np.arange(1,len(rtime),1)
slast = np.array(ds.sl[:,0])[indices]
sfirst = np.zeros_like(slast)
part1992 = indices[indices<limit]
part5yr = indices[indices>=limit]
sfirst[:len(part1992)] = np.array(ds.sf[0,:])[part1992]
sfirst[-len(part5yr):] = np.array(ds.sf[:,limit-1])[part5yr - limit]

In [ ]:
for var in rhs_list:
    terms[var] = np.array(terms[var])

In [ ]:
sums = np.zeros_like(terms['A'])
for var in rhs_list:
    sums+= np.array(terms[var])

In [ ]:
t = rtime[1:]

In [ ]:
NUMBER_OF_PARTICLE = NUMBER_OF_PARTICLE_domain_all

In [ ]:
to_output = xr.Dataset()
for var in terms.keys():
    to_output[var] = xr.DataArray(terms[var],dims = 'dummy')
to_output.to_zarr('lineplotprep/contr_domain_all')

In [ ]:
fig,axs = plt.subplots(2,2, figsize = (16,9),sharex = True)
fontsize = 7
axs = axs.ravel()
ylim = 0.3
small_ylim = 0.03

axs[0].plot(t,slast,label = 'final salinity')
axs[0].plot(t,sfirst, label = 'initial salinity',alpha = 0.5)
axs[0].plot(t,-sums[1:]/NUMBER_OF_PARTICLE, label = 'term contribution')
axs[0].plot(t,slast-sfirst+sums[1:]/NUMBER_OF_PARTICLE,label = 'closure',alpha = 0.2)
axs[0].legend(fontsize = fontsize)
axs[0].fill_betweenx([-ylim,ylim],t[0],t[1825],color = 'grey',alpha = 0.5)
axs[0].set_ylim([-ylim,ylim])
axs[0].set_xlim([t[0],t[-1]])

for i,group in enumerate([['E','A'],['dif_h','dif_v','F'],['e_ua','I']]):
    for var in group:
        axs[i+1].plot(ttt, -terms[var]/NUMBER_OF_PARTICLE,label = term_dic[var],color = color_dic[var])
    axs[i+1].legend(fontsize = fontsize)
    if i!=2:
        axs[i+1].fill_betweenx([-ylim,ylim],t[0],t[1825],color = 'grey',alpha = 0.5)
        axs[i+1].set_ylim([-ylim,ylim])
    else:
        axs[i+1].fill_betweenx([-small_ylim,small_ylim],t[0],t[1825],color = 'grey',alpha = 0.5)
        axs[i+1].set_ylim([-small_ylim,small_ylim])

plt.tight_layout()